In [1]:
!pip install langchain langchain_openai chromadb

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o",
    temperature = 0,
)

In [3]:
from langchain_community.document_loaders import CSVLoader
# CSV 파일 가져오기
loader = CSVLoader('d:/data/football.csv')
docs = loader.load()
docs[0] # 첫 번째 문서를 가져옴

Document(metadata={'source': 'd:/data/football.csv', 'row': 0}, page_content='sofifa_id: 158023\nshort_name: L. Messi\nage: 33\ndob: 1987-06-24\nheight_cm: 170\nweight_kg: 72\nnationality: Argentina\nclub_name: FC Barcelona\nleague_name: Spain Primera Division\noverall: 93\npotential: 93\nvalue_eur: 67500000\nwage_eur: 560000\nplayer_positions: RW, ST, CF\npreferred_foot: Left')

In [4]:
from langchain.text_splitter import CharacterTextSplitter

# 문서 분할 기준 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 문서를 청크로 나누기 (split_documents)
docs = text_splitter.split_documents(docs)

In [5]:
# OpenAI의 임베딩 모델 사용
from langchain_openai import OpenAIEmbeddings
embedding_function = OpenAIEmbeddings()

In [16]:
from langchain.vectorstores import Chroma
# docs를 embedding_function을 사용하여 벡터로 변환
db = Chroma.from_documents(docs, embedding_function) 

In [17]:
# 프롬프트 템플릿 정의
template = """당신은 축구에 대한 정보를 제공하는 챗봇입니다.

{context}

제공된 소스 데이터만 사용하여 고객의 질문에 답하세요.
모르겠다면 "잘 모르겠습니다."라고 답하세요.
친절하고 예의 바르며 전문적인 어조를 사용하세요.
답변은 간결하게 유지하세요.

질문:

답변: """

In [18]:
from langchain.prompts import PromptTemplate
# PromptTemplate을 사용하여 프롬프트 초기화
prompt = PromptTemplate(template=template, input_variables=["context"]) # input_variables=["context"]: 프롬프트에서 사용할 변수(context)를 정의

In [19]:
from langchain.chains import RetrievalQA
# RetrievalQA 체인에 사용할 프롬프트 설정
chain_type_kwargs = {"prompt": prompt} # 응답할 때 사용할 프롬프트(prompt)를 지정

# RetrievalQA 체인 생성
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    chain_type_kwargs=chain_type_kwargs, # 프롬프트 설정을 적용하여 응답 스타일 지정
)

In [20]:
import textwrap
# 긴 텍스트를 보기 좋게 정리하는 함수
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=80))) #  80자 단위로 텍스트를 자동 줄 바꿈

# 사용자 질의
query = "L. Messi의 소속팀은?"
response = chain.invoke(query)
# 답변 출력
response

{'query': 'L. Messi의 소속팀은?',
 'result': '질문을 제공해 주시면, 제공된 소스 데이터를 바탕으로 답변해 드리겠습니다.'}